In [291]:
MARKET = 'SNP'

In [292]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio
import datetime

from ib_insync import IB, util, Option, MarketOrder, Contract, Ticker, TickData
from typing import Callable, Coroutine, Union

In [293]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [294]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [295]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [296]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
files

['dfrq.pkl',
 'df_chains.pkl',
 'df_ohlcs.pkl',
 'df_opts.pkl',
 'df_opt_margins.pkl',
 'df_opt_prices.pkl',
 'df_symlots.pkl',
 'df_unds.pkl',
 'df_und_margins.pkl',
 'df_und_prices.pkl',
 'qopts.pkl']

# Getting implied volatility and price of an underlying

In [297]:
# .Price
async def price(ib: IB, c, FILL_DELAY=5) -> pd.DataFrame:
    """Price coro. Use CONCURRENT=40, TMEOUT=8 for optimal executeAsync results"""

    if isinstance(c, tuple):
        c = c[0]
        
    tick = ib.reqMktData(c, genericTickList='106')
    
    await asyncio.sleep(FILL_DELAY)

    try:
        dfpr = util.df([tick])
        dfpr['symbol'] = c.symbol
        dfpr['localSymbol'] = c.localSymbol
        dfpr['conId'] = c.conId
        dfpr['contract'] = c
        dfpr['strike'] = c.strike
        dfpr['expiry'] = c.lastTradeDateOrContractMonth
        dfpr['right'] = c.right
        dfpr['bid'] = dfpr.bid
        dfpr['ask'] = dfpr.ask
        dfpr['close'] = dfpr.close
        dfpr['last'] = dfpr['last']
        dfpr['price'] = dfpr["last"].combine_first(dfpr["close"])
        dfpr['time'] = dfpr.time
        dfpr['model'] = dfpr.modelGreeks
        if dfpr.model[0] is None:
            dfpr['iv'] = dfpr.impliedVolatility
        else:
            dfpr['iv'] = dfpr.model[0].impliedVol
        
    except AttributeError as e:
        
        print(f'\nError in {c.localSymbol}: {e}. df will be empty!\n')

        # return empty price df
        dfpr = pd.DataFrame([]) # initialize empty df
        
        dfpr['symbol'] = c.symbol
        dfpr['localSymbol'] = c.localSymbol
        dfpr['conId'] = c.conId
        dfpr['contract'] = c
        dfpr['strike'] = c.strike
        dfpr['expiry'] = c.lastTradeDateOrContractMonth
        dfpr['right'] = c.right
        dfpr['bid'] = np.nan
        dfpr['ask'] = np.nan
        dfpr['close'] = np.nan
        dfpr['last'] = np.nan
        dfpr['price'] = np.nan
        dfpr['iv'] = np.nan
        dfpr['time'] = np.nan
        dfpr['model'] = None

    cols = ["symbol", "conId", "strike", "expiry", "right", "contract", 
            "time", "model", "bid", "ask", "close", "last", "price", "iv"]
    
    ib.cancelMktData(c)
        
    return dfpr[cols]

## Single contract with reqMktData

In [298]:
# ct = Contract(symbol='INTC', secType='STK', exchange='SMART', currency='USD')
und_ct = df_unds.contract.sample(1).iloc[0]
und_ct

Contract(secType='STK', conId=83975037, symbol='KMI', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='KMI', tradingClass='KMI')

In [299]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfpr_und = ib.run(price(ib, und_ct))
dfpr_und

Wall time: 5.28 s


,symbol,conId,strike,expiry,right,contract,time,model,bid,ask,close,last,price,iv
0,KMI,83975037,0.0,,,"Contract(secType='STK', conId=83975037, symbol...",2020-11-13 10:24:13.581672+00:00,None,12.54,12.94,12.71,12.7,12.7,0.364997


In [300]:
%%time
opt_ct = df_opts.contract.sample(1).iloc[0]
with IB().connect(HOST, PORT, CID) as ib:
    dfpr_opt = ib.run(price(ib, opt_ct))
dfpr_opt

Wall time: 5.22 s


,symbol,conId,strike,expiry,right,contract,time,model,bid,ask,close,last,price,iv
0,MET,449226282,34.0,20201127,P,"Option(conId=449226282, symbol='MET', lastTrad...",2020-11-13 10:24:19.026629+00:00,None,-1.0,-1.0,0.0,NaN,0.0,NaN


# Multiple contracts

In [301]:
und_cts = df_unds.contract.to_list()

In [302]:
from engine import executeAsync, save_df

In [303]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfpr = ib.run(executeAsync(ib, price, und_cts, save_df, CONCURRENT=40, TIMEOUT=8))


Done price for ['SO0.0..', 'CBOE0.0..'] 40 out of 261. Pending ['OEX0.0..', 'BIIB0.0..']

Done price for ['BBY0.0..', 'KLAC0.0..'] 80 out of 261. Pending ['AVGO0.0..', 'CRM0.0..']

Done price for ['OKE0.0..', 'CHTR0.0..'] 120 out of 261. Pending ['WFC0.0..', 'AFL0.0..']

Done price for ['KMB0.0..', 'IP0.0..'] 160 out of 261. Pending ['ACN0.0..', 'AMAT0.0..']

Done price for ['DAL0.0..', 'ALXN0.0..'] 200 out of 261. Pending ['LLY0.0..', 'YINN0.0..']

Done price for ['HPE0.0..', 'TMUS0.0..'] 240 out of 261. Pending ['TGT0.0..', 'HD0.0..']

Done price for ['IWM0.0..', 'HD0.0..'] 261 out of 261. Pending []
Wall time: 39.6 s
